# XML Agent

Some language models (like Anthropic's Claude) are particularly good at reasoning/writing XML. This goes over how to use an agent that uses XML when prompting. 

In [22]:
from langchain import hub
from langchain.agents import AgentExecutor, create_xml_agent
from langchain_community.chat_models import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults

## Initialize the tools

We will initialize the tools we want to use

In [2]:
tools = [TavilySearchResults(max_results=1)]

## Create Agent

In [3]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")

In [4]:
print(prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between <final_answer></final_answer>. For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\nBegin!\n\nPrevious Conversation:\n{chat_history}\n\nQuestion: {input}\n{agent_scratchpad}"))]


In [5]:
# Choose the LLM that will drive the agent
llm = ChatAnthropic(model="claude-2")

# Construct the XML agent
agent = create_xml_agent(llm, tools, prompt)

## Run the Agent

In [6]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
 <tool>tavily_search_results_json</tool><tool_input>what is LangChain

Connection error caused failure to post https://dev.api.langchain.plus/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='dev.api.langchain.plus', port=443): Max retries exceeded with url: /runs (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)')))"))
Connection error caused failure to patch https://dev.api.langchain.plus/runs/bf02940c-83a6-45dc-90a4-afe87ba95fc6  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. SSLError(MaxRetryError("HTTPSConnectionPool(host='dev.api.langchain.plus', port=443): Max retries exceeded with url: /runs/bf02940c-83a6-45dc-90a4-afe87ba95fc6 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)')))"))


[{'url': 'https://www.commandbar.com/blog/langchain-projects', 'content': 'What Is LangChain?  What Is The Role Of LangChain?  Guide to LangChain resource.  At its core, LangChain boasts modular components for efficient language model operations and off-the-shelf chainsLatest Update December 5, 2023 In the rapidly evolving world of technology, one tool is making waves and transforming the landscape: LangChain. This powerful tool is revolutionizing the way developers build applications using large language models.'}] <final_answer>
LangChain is a tool that enables developers to build applications using large language models in a modular and efficient way. It features reusable components for common language model operations, making it easier to create "chains" of models to power complex apps. Some key things to know about LangChain:

- It facilitates the use of large language models like GPT-3 for developers by handling tedious tasks like prompt engineering and chaining models together.


{'input': 'what is LangChain?',
 'output': '\nLangChain is a tool that enables developers to build applications using large language models in a modular and efficient way. It features reusable components for common language model operations, making it easier to create "chains" of models to power complex apps. Some key things to know about LangChain:\n\n- It facilitates the use of large language models like GPT-3 for developers by handling tedious tasks like prompt engineering and chaining models together.\n\n- It has modular components so developers can mix and match different models and tools instead of building everything from scratch.\n\n- It aims to make building with language models more accessible and production-ready.\n\n- LangChain helps manage costs, efficiency, and scaling when using expensive language model APIs.\n\nSo in summary, LangChain is an open-source library focused on streamlining the process of leveraging large language models to build useful applications. It abstr

## Use LangChain Expression Language

We will first show how to create this agent using LangChain Expression Language

In [8]:
from langchain import hub
from langchain.agents.format_scratchpad import format_xml
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain.tools.render import render_text_description

In [9]:
prompt = hub.pull("hwchase17/xml-agent-convo")

In [10]:
prompt = prompt.partial(
    tools=render_text_description(tools),
)

In [12]:
llm_with_stop = llm.bind(stop=["</tool_input>"])

In [18]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_xml(x["intermediate_steps"]),
        # Give it a default
        "chat_history": lambda x: x.get("chat_history", ""),
    }
    | prompt
    | llm_with_stop
    | XMLAgentOutputParser()
)

In [19]:
from langchain.agents import AgentExecutor

In [20]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
 <tool>tavily_search_results_json</tool><tool_input>what is LangChain[{'url': 'https://www.commandbar.com/blog/langchain-projects', 'content': 'What Is LangChain?  What Is The Role Of LangChain?  Guide to LangChain resource.  At its core, LangChain boasts modular components for efficient language model operations and off-the-shelf chainsLatest Update December 5, 2023 In the rapidly evolving world of technology, one tool is making waves and transforming the landscape: LangChain. This powerful tool is revolutionizing the way developers build applications using large language models.'}] <final_answer>
LangChain is a tool for building modular pipelines that connect language models and other components. It allows developers to efficiently chain together different models and data sources to create custom AI applications.

Some key things to know about LangChain:

- It provides a framework for connecting modular components like language models (e.g. GPT

{'input': 'what is LangChain?',
 'output': '\nLangChain is a tool for building modular pipelines that connect language models and other components. It allows developers to efficiently chain together different models and data sources to create custom AI applications.\n\nSome key things to know about LangChain:\n\n- It provides a framework for connecting modular components like language models (e.g. GPT-3), search engines, databases, etc.\n\n- It makes it easier to build pipelines and workflows that leverage large language models. \n\n- It aims to simplify and streamline the process of creating AI apps with language models.\n\n- By handling coordination and data flow behind the scenes, LangChain frees up developers to focus on the functionality and UX of their applications.\n\nSo in summary, LangChain is an open-source library for efficiently leveraging large language models to build custom AI tools and products. It streamlines the process by providing infrastructure and abstractions for